[Source code](https://github.com/trangnv/starknet-watcher/) using [Checkpoint](https://checkpoint.fyi/#/) as indexer to get Starknet data

In [67]:
from datetime import datetime
last_query_block = creates_data_df['created_at_block'].iloc[-1]
time_utc = datetime.utcfromtimestamp(creates_data_df['created_at'].iloc[-1]).strftime('%Y-%m-%d %H:%M:%S')
print(f'Last query block: {last_query_block}, at {time_utc} GMT')

Last query block: 38793, at 2023-04-14 13:44:48 GMT


## Query/save/load data

In [24]:
import pandas as pd
import json
from tabulate import tabulate
from utils.query import query_account_creates, query_account_upgrades
# Query and save data
# query_account_creates()
# query_account_upgrades()

# Load data
with open(f'data/account_creates_up_to_block_38792.json') as f:
    creates_data = json.load(f)
creates_data_df = pd.DataFrame(creates_data['account_creates']).drop_duplicates(subset=['id']).reset_index(drop=True)
creates_data_df['datetime'] = pd.to_datetime(creates_data_df['created_at'], unit='s')

print("Account creates table (first 5 rows):")
print(tabulate(creates_data_df.head(5), headers='keys', tablefmt='psql'))
print(f'Shape: {creates_data_df.shape}')
print('\n\n')

# ####
with open(f'data/account_upgrades_up_to_block_38787.json') as f:
    upgrades_data = json.load(f)
upgrades_data_df = pd.DataFrame(upgrades_data['account_upgrades']).drop_duplicates(subset=['id']).reset_index(drop=True)
upgrades_data_df['datetime'] = pd.to_datetime(upgrades_data_df['created_at'], unit='s')
print("Account upgrades table (first 5 rows):")
print(tabulate(upgrades_data_df.head(5), headers='keys', tablefmt='psql'))
print(f'Shape: {upgrades_data_df.shape}')

# datetime are in GMT

Account creates table (first 5 rows):
+----+-------------------------------------------------------------------+-------------------------------------------------------------------+--------------+--------------------+---------------------+
|    | id                                                                | from_address                                                      |   created_at |   created_at_block | datetime            |
|----+-------------------------------------------------------------------+-------------------------------------------------------------------+--------------+--------------------+---------------------|
|  0 | 0x2b5f4af6bc5cfb032c83baef17ca99ee46f2d60997d5ad78d75efefa9617139 | 0x2a7f765dc8a9030e3ec393f51d5b0244a21ae55b71485f1142474b3c254537e |   1655372577 |               2710 | 2022-06-16 09:42:57 |
|  1 | 0x3ef53cf0fbad0edde37c1ee24ebc57433f39728dcaa9dfb2d2b365d3f9e8e6b | 0x179b1211aed20097bbba0369088db3d30ece0550015c08ea7e521bba99e76d2 |   1655380283 | 

## Analysis

In [22]:
#| code-fold: False
# Just to make sure all the addresses that send `create_account` transactions are unique
creates_data_df['from_address'].unique().shape[0] == creates_data_df.shape[0]

True

### Events count by day
This tells a lot about when users are interested in getting into the network.

In [63]:
import plotly.graph_objects as go
fig = go.Figure()

s = creates_data_df['datetime']
df = s.groupby(s.dt.floor('d')).size().reset_index(name='count')
fig.add_trace(go.Scatter(x=df['datetime'], y=df['count'],
                    mode='markers',
                    name='account_created'))

s = upgrades_data_df['datetime']
df = s.groupby(s.dt.floor('d')).size().reset_index(name='count')
fig.add_trace(go.Scatter(x=df['datetime'], y=df['count'],
                     mode='markers',
                     name='account_upgraded'))

# Edit the layout
fig.update_layout(title='Daily number of events',
                   xaxis_title='Day',
                   autosize=True,
    )
# fig.add_vline(x='2023-03-23', annotation_text='ARB', line_width=2, line_dash="dash", line_color="green")
fig.add_vrect(x0="2023-03-22", x1="2023-03-24", row="all", col=1,
              annotation_text="ARB airdrop", annotation_position="top right",
              fillcolor="green", opacity=0.25, line_width=0)

fig.show()

### Around the clock
Not much information, just when (most) users sleep.

In [72]:
fig = go.Figure()

s = creates_data_df['datetime']
df = s.groupby(s.dt.floor('h')).size().reset_index(name='count')
_df = df.groupby([df['datetime'].dt.hour])['count'].sum().reset_index(name='count')
fig.add_trace(go.Bar(x=_df['datetime'], y=_df['count'], name='account_created'))

s = upgrades_data_df['datetime']
df = s.groupby(s.dt.floor('h')).size().reset_index(name='count')
_df = df.groupby([df['datetime'].dt.hour])['count'].sum().reset_index(name='count')
fig.add_trace(go.Bar(x=_df['datetime'], y=_df['count'], name='account_upgraded'))

fig.update_layout(title='Number of events around the clock',
                   xaxis_title='Hour (UTC)',
                   autosize=True,
    )
fig.show()

### Account implementations
Or contract class which is used by the account.
- This is not the whole picture, as `account_created` event does not contain the implementation.
- But by far [ArgentAccount](https://starkscan.co/class/0x033434ad846cdd5f23eb73ff09fe6fddd568284a0fb7d1be20ee482f044dabe2) is the most implementation that accounts are upgraded to.

In [85]:
#| code-fold: False
_df = upgrades_data_df['implementation'].value_counts().rename_axis('implementation').reset_index(name='counts')
print(tabulate(_df, headers='keys', tablefmt='psql'))

+----+-------------------------------------------------------------------+----------+
|    | implementation                                                    |   counts |
|----+-------------------------------------------------------------------+----------|
|  0 | 0x33434ad846cdd5f23eb73ff09fe6fddd568284a0fb7d1be20ee482f044dabe2 |    72987 |
|  1 | 0x1a7820094feaf82d53f53f214b81292d717e7bb9a92bb2488092cd306f3993f |    11331 |
|  2 | 0x1bd7ca87f139693e6681be2042194cf631c4e8d77027bf0ea9e6d55fc6018ac |     2225 |
|  3 | 0x69577e6756a99b584b5d1ce8e60650ae33b6e2b13541783458268f07da6b38a |      295 |
|  4 | 0x25319262b1256b464b0a7b7678fc3fa1519015cee5a559a22dfa3e52bdef2a8 |        1 |
|  5 | 0x4507edaf81d13231eddb027922a7d535a5c0bd647d953b7f2fa84926d659ec9 |        1 |
|  6 | 0x41e620242076112068698358a60b4f55ba647191663e05eff434a8c84e63cab |        1 |
|  7 | 0x1ce9e67ecfbe0d645db2425ba00bcef7c29a73d87e74d3d96bfaffc237be1ec |        1 |
|  8 | 0x7c3a28432b38cdc92f2d897227d2a125857edbf2c3d4b